In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import random
import os
import matplotlib.pyplot as plt

# Фиксируем seed для воспроизводимости
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(42)
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
print(f"Device: {device}")

Device: mps


# Задание 1
В этом задании вам нужно будет подобрать архитектуру нейронной сети, чтобы пробить порог в 80% accuracy

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)


In [ ]:
# ==========================================
# ВАША АРХИТЕКТУРА (TODO)
# ==========================================

class MySuperCNN(nn.Module):
    def __init__(self):
        super(MySuperCNN, self).__init__()
        
        # ------------------------------------------------------------------
        # TODO: Определите слои вашей модели здесь
        # Подсказка: Используйте nn.Sequential, nn.Conv2d, nn.BatchNorm2d, nn.ReLU, nn.MaxPool2d
        # Пример: 
        # self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        # ------------------------------------------------------------------
        
        pass 

    def forward(self, x):
        # ------------------------------------------------------------------
        # TODO: Опишите прямой проход
        # Не забудьте про flatten перед полносвязными слоями: x = x.view(x.size(0), -1)
        # ------------------------------------------------------------------
        
        return x


In [ ]:
model = MySuperCNN().to(device)

num_params = sum(p.numel() for p in model.parameters())
print(f"Количество параметров: {num_params:,}")

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train_loop(model, epochs=10):
    best_acc = 0.0
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        
        for i, (inputs, labels) in enumerate(trainloader):
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        # Валидация в конце эпохи
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in testloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        val_acc = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs} | Loss: {running_loss/len(trainloader):.4f} | Val Acc: {val_acc:.2f}%")
        
        if val_acc > best_acc:
            best_acc = val_acc

    return best_acc


In [ ]:
final_acc = train_loop(model, epochs=10) 

print(f"\nИтоговая точность: {final_acc:.2f}%")

In [ ]:
# Проверка результата
assert final_acc > 80.0, "Точность ниже 80%. Попробуйте добавить слоев, BatchNorm или увеличить ширину сети!"
print("Отличная работа! Тест пройден.")

# Задание 2

In [ ]:
class FixedCNN(nn.Module):
    def __init__(self):
        super(FixedCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.MaxPool2d(2, 2), # 16x16
            
            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.MaxPool2d(2, 2), # 8x8
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 8 * 8, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# Настройки обучения
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.CrossEntropyLoss()
# Фиксированная статистика нормализации для CIFAR-10
stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))

In [ ]:
# Функция для запуска эксперимента
def run_experiment(name, train_transform, epochs=10):
    print(f"\n=== Запуск эксперимента: {name} ===")
    
    # Тестовый трансформ всегда фиксирован (только нормализация)
    test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(*stats)
    ])
    
    # Загрузка данных
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
    
    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)
    
    # Реинициализация модели
    model = FixedCNN().to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    
    train_accs = []
    val_accs = []
    
    for epoch in range(epochs):
        model.train()
        correct_train = 0
        total_train = 0
        
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            _, predicted = outputs.max(1)
            total_train += labels.size(0)
            correct_train += predicted.eq(labels).sum().item()
            
        train_acc = 100 * correct_train / total_train
        train_accs.append(train_acc)
        
        # Validation
        model.eval()
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for inputs, labels in testloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = outputs.max(1)
                total_val += labels.size(0)
                correct_val += predicted.eq(labels).sum().item()
        
        val_acc = 100 * correct_val / total_val
        val_accs.append(val_acc)
        
        print(f"Epoch {epoch+1}: Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")
        
    return train_accs, val_accs

In [ ]:
# ЗАДАЙТЕ ТРИ СТРАТЕГИИ АУГМЕНТАЦИИ

# Стратегия 1: Baseline (ТолькоToTensor + Normalize). Должен быть сильный переобуч.
transforms_baseline = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(*stats)
])

# Стратегия 2: "Вредные" аугментации. 
# Попробуйте то, что ломает семантику (RandomVerticalFlip, слишком сильный Jitter, Invert и т.д.)
transforms_bad = transforms.Compose([
    # TODO: Добавьте сюда плохие аугментации
    transforms.RandomVerticalFlip(p=1.0), # Пример: переворачиваем машины вверх колесами
    transforms.ToTensor(),
    transforms.Normalize(*stats)
])

# Стратегия 3: "Идеальные" аугментации.
# Попробуйте RandomHorizontalFlip, RandomSizedCrop, RandomRotation и т.д.
transforms_best = transforms.Compose([
    # TODO: Добавьте сюда хорошие аугментации
    # transforms.RandomHorizontalFlip(),
    # ...
    transforms.ToTensor(),
    transforms.Normalize(*stats)
])

## ВИЗУАЛИЗАЦИЯ (Обязательно посмотрите, что вы скармливаете сети!)

In [ ]:
def show_aug(transform, title):
    # Берем чистую картинку
    raw_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True)
    img = raw_set.data[0] # Картинка numpy array
    img_pil = transforms.ToPILImage()(img)
    
    fig, axes = plt.subplots(1, 5, figsize=(15, 3))
    fig.suptitle(title, fontsize=16)
    for i in range(5):
        # Применяем трансформ (нужно убрать Normalize для визуализации, чтобы цвета не поплыли)
        # Это хак для визуализации, в реальном обучении Normalize нужен
        aug_img = transform(img_pil)
        
        # Если трансформ вернул Tensor, переводим обратно для показа
        if isinstance(aug_img, torch.Tensor):
             # Denormalize вручную для показа, если там был Normalize
             aug_img = aug_img.permute(1, 2, 0).numpy()
             aug_img = (aug_img * stats[1]) + stats[0] # un-normalize
             aug_img = np.clip(aug_img, 0, 1)
             
        axes[i].imshow(aug_img)
        axes[i].axis('off')
    plt.show()

# Эта функция покажет, как transforms_best меняют одну и ту же лягушку
# (Уберите нормализацию из transforms_best временно или создайте копию для визуализации без неё)
# show_aug(transforms_best, "My Best Augmentations")

In [ ]:
# 1. Запуск Baseline
train_base, val_base = run_experiment("Baseline (No Augs)", transforms_baseline, epochs=10)

# 2. Запуск Bad
train_bad, val_bad = run_experiment("Bad Augmentations", transforms_bad, epochs=10)

# 3. Запуск Best
train_best, val_best = run_experiment("Best Augmentations", transforms_best, epochs=10)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(val_base, label='Baseline Val', linestyle='--')
plt.plot(val_bad, label='Bad Augs Val', linestyle=':')
plt.plot(val_best, label='Best Augs Val', linewidth=2)
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
plt.title('Битва Аугментаций: Валидация')
plt.legend()
plt.grid()
plt.show()

print(f"Final Best Val Acc: {val_best[-1]:.2f}%")